## Voting for Classification 

### We will be using the IRIS dataset 


In [1]:
#Lets import the libraries and datasets 
import numpy as np

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

import statistics as stat

In [2]:
#Lets load the dataset
iris = datasets.load_iris()
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [3]:
X = iris.data[:, [0, 2]]
y = iris.target

In [4]:
#Lets check the shape of our data 
print(X.shape)
print(y.shape)

(150, 2)
(150,)


In [5]:
#Output classes
list(iris.target_names)

['setosa', 'versicolor', 'virginica']

In [6]:
#Lets split our data in train and test 
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [7]:
#Lets check the shape 
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(112, 2)
(112,)
(38, 2)
(38,)


In [8]:
#We initialize the models
model1 = DecisionTreeClassifier(max_depth=4)
model2 = KNeighborsClassifier(n_neighbors=7)
model3 = SVC(kernel='rbf', probability=True)

In [9]:
#We then fit the models 
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### Lets look at the accuracy for each model 

In [10]:
#The accuracy score for the first model is
score1=model1.score(X_test, y_test)
print(score1)

0.9736842105263158


In [11]:
#The accuracy score for the second model is
score2=model2.score(X_test, y_test)
print(score2)

1.0


In [12]:
#The accuracy score for the third model is
score3=model3.score(X_test, y_test)
print(score3)

0.9736842105263158


### Lets combine the models using voting in python 

In [13]:
#Lets find out what each model predicts 
pred1=model1.predict(X_test)
pred2=model2.predict(X_test)
pred3=model3.predict(X_test)

#Lets take the vote using the mode function 
final_pred = np.array([])
for i in range(0,len(X_test)):
    final_pred = np.append(final_pred, stat.mode([pred1[i], pred2[i], pred3[i]]))

### This is also called as a hard voting since we are using the model predictions to get the final combined prediction 

In [14]:
total=np.sum(y_test==final_pred)

print("Accuracy:",total,"/",len(final_pred),"* 100 =","{0:.3f}".format(total/len(final_pred)*100),"%")

Accuracy: 37 / 38 * 100 = 97.368 %


### Soft voting in python

In [15]:
spred1=model1.predict_proba(X_test)
spred2=model2.predict_proba(X_test)
spred3=model3.predict_proba(X_test)

finalpred=(spred1+spred2+spred3)/3

In [16]:
final_classes = finalpred.argmax(axis=-1) 
final_classes

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0])

In [17]:
total=np.sum([y_test[i]==final_classes[i] for i in range(len(final_classes))])

print("Accuracy:",total,"/",len(final_classes),"* 100 =","{0:.3f}".format(total/len(final_classes)*100),"%")

Accuracy: 37 / 38 * 100 = 97.368 %


### We can also use VotingClassifier from sklearn to combine the models

In [18]:
emodel = VotingClassifier(estimators=[('dt', model1), ('knn', model2),
                                    ('svc', model3)],
                        voting='soft')
emodel.fit(X_train, y_train)


VotingClassifier(estimators=[('dt', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0....',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [19]:
#The accuracy score for the ensemble model is
escore=emodel.score(X_test, y_test)
print(escore)

0.9736842105263158


/home/arish/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
